In [1]:

# sphinx_gallery_thumbnail_number = 2

# Authors: Robert Luke <mail@robertluke.net>
#
# License: BSD (3-clause)

# Import common libraries
from collections import defaultdict
from copy import deepcopy
from itertools import compress
from pprint import pprint

# Import Plotting Library
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
import os
import pywt
import mne_nirs
import mne



# Import StatsModels
import statsmodels.formula.api as smf
from mne import Epochs, events_from_annotations, set_log_level
from mne.preprocessing.nirs import (
    beer_lambert_law,
    optical_density,
    scalp_coupling_index,
    temporal_derivative_distribution_repair,
    
)

# Import MNE processing
from mne.viz import plot_compare_evokeds

# Import MNE-BIDS processing
from mne_bids import BIDSPath, read_raw_bids


# Import MNE-NIRS processing
from mne_nirs.channels import get_long_channels, picks_pair_to_idx
from mne_nirs.datasets import fnirs_motor_group
from mne_nirs.signal_enhancement import (enhance_negative_correlation, short_channel_regression)
from mne_nirs.channels import (get_long_channels,
                               get_short_channels,
                               picks_pair_to_idx)

from collections import defaultdict
import numpy as np
from itertools import compress
from sklearn.decomposition import PCA

from mne_nirs.experimental_design import make_first_level_design_matrix
from mne_nirs.statistics import run_glm, statsmodels_to_results
from nilearn.plotting import plot_design_matrix

# Set general parameters
set_log_level("WARNING")  # Don't show info, as it is repetitive for many subjects

In [2]:
# cropping the signal before sci calculation
def preprocessing_glm_ROI(bids_path, subject_id, session_id, id):
    # Read data with annotations in BIDS format
    raw_intensity = read_raw_bids(bids_path=bids_path, verbose=False)
    # check if coordinates of the channels
    
    
    #print(raw_intensity.ch_names)

    
    # Get event timings
    #print("Extracting event timings...")
    Breaks, _ = mne.events_from_annotations(raw_intensity, {'Xstart': 4, 'Xend': 5})
    AllEvents, _ = mne.events_from_annotations(raw_intensity)
    Breaks = Breaks[:, 0] / raw_intensity.info['sfreq']
    LastEvent = AllEvents[-1, 0] / raw_intensity.info['sfreq']
    
    if len(Breaks) % 2 == 0:
        raise ValueError("Breaks array should have an odd number of elements.")
    
    original_duration = raw_intensity.times[-1] - raw_intensity.times[0]
    #print(f"Original duration: {original_duration:.2f} seconds")
    
    # Cropping dataset
    #print("Cropping the dataset...")
    cropped_intensity = raw_intensity.copy().crop(Breaks[0], Breaks[1])
    for j in range(2, len(Breaks) - 1, 2):
        block = raw_intensity.copy().crop(Breaks[j], Breaks[j + 1])
        cropped_intensity.append(block)
    cropped_intensity.append(raw_intensity.copy().crop(Breaks[-1], LastEvent + 15.25))
    
    cropped_duration = cropped_intensity.times[-1] - cropped_intensity.times[0]
    #print(f"Cropped duration: {cropped_duration:.2f} seconds")
    
    if cropped_duration >= original_duration:
        print(f"WARNING: Cropping did not reduce duration!")
    
    raw_intensity_cropped = cropped_intensity.copy()

    

    
    # Remove break annotations
    print("Removing break annotations for the orginal raw...")
    raw_intensity.annotations.delete(np.where(
        (raw_intensity.annotations.description == 'Xstart') | 
        (raw_intensity.annotations.description == 'Xend') | 
        (raw_intensity.annotations.description == 'BAD boundary') | 
        (raw_intensity.annotations.description == 'EDGE boundary')
    )[0])
    
    print("Removing break annotations for the cropped raw...")
    raw_intensity_cropped.annotations.delete(np.where(
        (raw_intensity_cropped.annotations.description == 'Xstart') | 
        (raw_intensity_cropped.annotations.description == 'Xend') | 
        (raw_intensity_cropped.annotations.description == 'BAD boundary') | 
        (raw_intensity_cropped.annotations.description == 'EDGE boundary')
    )[0]) 
    
    
    #downsampling the signal
    #raw_intensity_cropped.resample(3)
    
    # Convert signal to optical density and determine bad channels
    raw_od = optical_density(raw_intensity)
    raw_od_cropped = optical_density(raw_intensity_cropped)
    
    # get the total number of short channels
    short_chs = get_short_channels(raw_od)
    tot_number_of_short_channels = len(short_chs.ch_names)
    

    # sci calculated
    sci = scalp_coupling_index(raw_od_cropped, l_freq=0.7, h_freq=1.45)
    #sci = scalp_coupling_index(raw_od_cropped, h_freq=1.35)
    bad_channels= list(compress(raw_od.ch_names, sci < 0.8))
    
    if len(bad_channels) > 55:
        print(f"❌ Too many bad channels ({len(bad_channels)}). Excluding subject from analysis.")
        return None
    
    raw_od.info["bads"] = bad_channels
    raw_intensity_cropped.info["bads"] = bad_channels
    
    print(f"Bad channels: {raw_od.info['bads']}")
    # print the number of bad channels
    print(f"Number of bad channels: {len(raw_od.info['bads'])}")
    
    # Remove bad channels
   
    
    
    raw_od = temporal_derivative_distribution_repair(raw_od)
    raw_od_cropped = temporal_derivative_distribution_repair(raw_od_cropped)

    
     # Get long channels
    long_chs = get_long_channels(raw_od)
    bad_long_chs = long_chs.info["bads"]
    
    """ print(f"Number of all (good and bad) short channels: {tot_number_of_short_channels}")
    print(f"Number of bad long channels: {len(bad_long_chs)}")
    print(f"Number of long and short bad channels: {len(bad_channels)}") """
    # print the number of short bad channels
    len_bad_short_chs = len(bad_channels) - len(bad_long_chs)
    #print(f"Number of bad short channels: {len_bad_short_chs}")
    
    # Determine if there are short channels
    # print the number of short bad channels
    len_bad_short_chs = len(bad_channels) - len(bad_long_chs)
    num_good_short_channels = tot_number_of_short_channels - len_bad_short_chs
    # Print diagnostics
    print(f"Number of all (good and bad) short channels: {tot_number_of_short_channels}")
    print(f"Number of bad long channels: {len(bad_long_chs)}")
    print(f"Number of bad short channels: {len_bad_short_chs}")
    print(f"✅ Number of good short channels: {num_good_short_channels}")
    #print(f"Number of bad short channels: {len_bad_short_chs}")
    
    # Determine if there are short channels
    if num_good_short_channels < 4:
        print("❌ No short channels found. Skipping the subject.")
        return None, None, None, None, None # Keep the data unchanged
    else:
        #print("Applying short-channel regression.")
        raw_od_corrected = short_channel_regression(raw_od)
        #raw_od_corrected=raw_od.copy()
        # drop the bad channels
        #raw_od_corrected.drop_channels(bad_channels)
        
        # interpolate the bad channels
        #raw_od_corrected.interpolate_bads()
        
    # short-channel regression subtracts a scaled version of the signal obtained from the nearest short channel from the signal obtained from the long channel. 

    raw_haemo = beer_lambert_law(raw_od_corrected, ppf=0.1)
    
    #raw_haemo = get_long_channels(raw_haemo, min_dist=0.02, max_dist=0.04) # max_dist 40mm
    raw_haemo = get_long_channels(raw_haemo, min_dist=0.02) 
    
    
   
    

    # Convert to haemoglobin and filter
    
    # check the ppf
    
    #raw_haemo.plot_psd(average=True, show=True)
    
    """ raw_haemo = raw_haemo.filter(
    l_freq=0.01, h_freq=0.7, method="fir", fir_design="firwin", verbose=False,
    h_trans_bandwidth=0.3, l_trans_bandwidth=0.005) """
    
    # improved filter
    """ raw_haemo = raw_haemo.filter(
    l_freq=0.05, h_freq=0.2, method="fir", fir_design="firwin", verbose=False,
    h_trans_bandwidth=0.01, l_trans_bandwidth=0.01) """
    
    #raw_haemo.plot_psd(average=True, show=True)

    raw_haemo = raw_haemo.filter(l_freq = None, h_freq = 0.2,  
                                 method="iir", iir_params =dict(order=5, ftype='butter'))
     #high-pass
    raw_haemo= raw_haemo.filter(l_freq =  0.05, h_freq = None, method="iir", iir_params =dict(order=5, ftype='butter'))
        
    raw_stim = raw_haemo.copy()
    #raw_stim.annotations.delete(raw_stim.annotations.description == 'Control') 
    
    isis, names = mne_nirs.experimental_design.longest_inter_annotation_interval(raw_stim)
    # Create a design matrix
    design_matrix = make_first_level_design_matrix(raw_stim,
                                                drift_model='cosine',
                                                high_pass=1/(2*max(isis)),  # Must be specified per experiment
                                                hrf_model='spm',
                                                stim_dur=5.125)
    
    
    # Run GLM
    glm_est = run_glm(raw_haemo, design_matrix)

    # Define ROI channel pairs
    left = [[4, 2], [4, 3], [5, 2], [5, 3], [5, 4], [5, 5]]
    right = [[10, 9], [10, 10], [10, 11], [10, 12], [11, 11], [11, 12]]
    back = [[6, 6], [6, 8], [7, 6], [7, 7], [7, 8], [8, 7], [8, 8], [9, 8]]
    front = [[1, 1], [2, 1], [3, 1], [3, 2], [12, 1]]
    noise= [[10, 11], [5, 3], [11, 11], [10, 9], [10, 12], [10, 10]]
    speech= [[10, 11], [10, 10], [10, 12], [11, 12], [1, 1], [10, 9], [8, 7], [7, 7], [8, 8]]
    common= [[10, 11], [10, 9], [10, 12], [10, 10]]
    only_noise= [[5, 3], [11, 11]]
    only_speech= [ [11, 12], [1, 1], [10, 9], [8, 7], [7, 7], [8, 8]]
    

    # Generate index picks for each ROI
    roi_picks = dict(
        Left= picks_pair_to_idx(raw_haemo, left, on_missing="ignore"),
        Right= picks_pair_to_idx(raw_haemo, right, on_missing="ignore"),
        Back= picks_pair_to_idx(raw_haemo, back, on_missing="ignore"),
        Front= picks_pair_to_idx(raw_haemo, front, on_missing="ignore"),
        Noise= picks_pair_to_idx(raw_haemo, noise, on_missing="ignore"),
        Speech= picks_pair_to_idx(raw_haemo, speech, on_missing="ignore"),
        Common= picks_pair_to_idx(raw_haemo, common, on_missing="ignore"),
        OnlyNoise= picks_pair_to_idx(raw_haemo, only_noise, on_missing="ignore"),
        OnlySpeech= picks_pair_to_idx(raw_haemo, only_speech, on_missing="ignore")
    )
    
    cha= glm_est.to_dataframe()
    
    roi= glm_est.to_dataframe_region_of_interest(
        roi_picks, design_matrix.columns, demographic_info=True
    )
    
    # Define left vs right tapping contrast
    contrast_matrix = np.eye(design_matrix.shape[1])
    basic_conts = dict(
        [(column, contrast_matrix[i]) for i, column in enumerate(design_matrix.columns)]
    )
    contrast_LvR = basic_conts["Noise"] - basic_conts["Speech"]

    # Compute defined contrast
    contrast = glm_est.compute_contrast(contrast_LvR)
    con = contrast.to_dataframe()

    # Add the participant sub to the dataframes
    roi["ID"] = cha["ID"] = con["ID"] = id
    
    roi["Subject"] = cha["Subject"] = con["Subject"] = subject_id
    
    # Add the session to the dataframes
    roi["session"] = cha["session"] = con["session"] = session_id

    # Convert to uM for nicer plotting below.
    cha["theta"] = [t * 1.0e6 for t in cha["theta"]]
    roi["theta"] = [t * 1.0e6 for t in roi["theta"]]
    con["effect"] = [t * 1.0e6 for t in con["effect"]]
    
    
    
    return raw_haemo, roi, cha, con
    

In [6]:

bids_root = r"C:\Datasets\Test-retest study\bids_dataset"
subject_list = sorted([d for d in os.listdir(bids_root) if d.startswith("sub-")])
subject_list = [s.replace("sub-", "") for s in subject_list]
#subject_list = subject_list[:1]  # Limit to first 3 subjects for testing

print("Detected subjects:", subject_list)
subjects_df = pd.DataFrame(columns=["subject", "session"])




#optode_subject_sessions = {optode: pd.DataFrame(columns=["subject", "session"]) for optode in optodes}

df_roi = pd.DataFrame()  # To store region of interest results
df_cha = pd.DataFrame()  # To store channel level results
df_con = pd.DataFrame()  # To store channel level contrast results
id = 0
# Loop through subjects and sessions
for sub in subject_list:
    for ses in range(1, 3):
        
        bids_path = BIDSPath(
                subject=f"{sub}",
                session=f"{ses:02d}",
                task="auditory",
                datatype="nirs",
                root=bids_root,
                suffix="nirs",
                extension=".snirf",
            )
            
        raw_haemo, roi, cha, con = preprocessing_glm_ROI(bids_path, sub, ses, id)
        if raw_haemo is None:
            print(f"⚠️ No data for Subject {sub}, Session {ses:02d}. Skipping...")
            continue
        else:
            
            # Append individual results to all participants
            df_roi = pd.concat([df_roi, roi], ignore_index=True)
            df_cha = pd.concat([df_cha, cha], ignore_index=True)
            df_con = pd.concat([df_con, con], ignore_index=True)
            id= id + 1
                


Detected subjects: ['01', '02', '03', '04', '05', '07', '08', '10', '11', '12', '13', '16', '17', '19', '21', '24']
Removing break annotations for the orginal raw...
Removing break annotations for the cropped raw...
Bad channels: ['S4_D14 785', 'S4_D14 830', 'S5_D15 785', 'S5_D15 830', 'S6_D6 785', 'S6_D6 830', 'S6_D8 785', 'S6_D8 830', 'S7_D6 785', 'S7_D6 830', 'S7_D8 785', 'S7_D8 830', 'S8_D17 785', 'S8_D17 830', 'S9_D8 785', 'S9_D8 830']
Number of bad channels: 16
Number of all (good and bad) short channels: 16
Number of bad long channels: 10
Number of bad short channels: 6
✅ Number of good short channels: 10
Removing break annotations for the orginal raw...
Removing break annotations for the cropped raw...
Bad channels: ['S2_D13 785', 'S2_D13 830', 'S7_D6 785', 'S7_D6 830', 'S7_D8 785', 'S7_D8 830', 'S8_D7 785', 'S8_D7 830', 'S8_D17 785', 'S8_D17 830', 'S9_D8 785', 'S9_D8 830', 'S10_D18 785', 'S10_D18 830', 'S12_D20 785', 'S12_D20 830']
Number of bad channels: 16
Number of all (goo

In [48]:
import pandas as pd
import statsmodels.formula.api as smf

# Step 1: Filter for session 1 and hbo only
ch_summary = df_cha.query("Condition in ['Control', 'Noise', 'Speech'] and Chroma == 'hbo' and session == 2").copy()

# Step 2: Ensure 'Condition' is categorical with 'Control' as the reference
ch_summary["Condition"] = pd.Categorical(
    ch_summary["Condition"], categories=["Control", "Noise", "Speech"], ordered=True
)

# Step 3: Fit per-channel models and collect results
results = []

for ch in ch_summary["ch_name"].unique():
    print(f"Fitting model for channel {ch}...")
    ch_data = ch_summary[ch_summary["ch_name"] == ch].copy()

    try:
        model = smf.mixedlm(
            "theta ~ Condition",  # Compare Noise & Speech vs Control
            ch_data,
            groups=ch_data["Subject"]
        ).fit(method="nm")

        # Extract parameters and p-values directly
        for param_name in model.params.index:
            if "Condition[T.Noise]" in param_name or "Condition[T.Speech]" in param_name:
                results.append({
                    "Channel": ch,
                    "Comparison": param_name,
                    "Beta": model.params[param_name],
                    "StdErr": model.bse[param_name],
                    "z": model.tvalues[param_name],
                    "p": model.pvalues[param_name]
                })

    except Exception as e:
        print(f"Model failed for channel {ch}: {e}")

# Step 4: Create DataFrame from results
significant = pd.DataFrame(results)

# Step 5: Filter for significance
significant = significant[significant["p"] < 0.05]

# View significant results
significant


Fitting model for channel S1_D1 hbo...
Fitting model for channel S2_D1 hbo...


c:\Users\sarab\anaconda3\envs\mne\Lib\site-packages\statsmodels\regression\mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
c:\Users\sarab\anaconda3\envs\mne\Lib\site-packages\statsmodels\regression\mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


Fitting model for channel S3_D1 hbo...
Fitting model for channel S3_D2 hbo...


c:\Users\sarab\anaconda3\envs\mne\Lib\site-packages\statsmodels\regression\mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


Fitting model for channel S4_D2 hbo...
Fitting model for channel S4_D3 hbo...


...

Fitting model for channel S9_D8 hbo...
Fitting model for channel S10_D9 hbo...
Fitting model for channel S10_D10 hbo...
Fitting model for channel S10_D11 hbo...
Fitting model for channel S10_D12 hbo...


c:\Users\sarab\anaconda3\envs\mne\Lib\site-packages\statsmodels\regression\mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
c:\Users\sarab\anaconda3\envs\mne\Lib\site-packages\statsmodels\regression\mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


Fitting model for channel S11_D11 hbo...
Fitting model for channel S11_D12 hbo...
Fitting model for channel S12_D1 hbo...


c:\Users\sarab\anaconda3\envs\mne\Lib\site-packages\statsmodels\regression\mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


,Channel,Comparison,Beta,StdErr,z,p
2,S2_D1 hbo,Condition[T.Noise],-1.333562,0.677513,-1.968320,0.049031
3,S2_D1 hbo,Condition[T.Speech],-1.867265,0.677513,-2.756060,0.005850
7,S3_D2 hbo,Condition[T.Speech],-2.429519,0.931356,-2.608584,0.009092
13,S5_D2 hbo,Condition[T.Speech],-2.019603,0.966973,-2.088583,0.036745
22,S6_D8 hbo,Condition[T.Noise],-1.474219,0.629866,-2.340527,0.019257
23,S6_D8 hbo,Condition[T.Speech],-2.244863,0.629866,-3.564033,0.000365
25,S7_D6 hbo,Condition[T.Speech],-3.607112,0.907098,-3.976540,0.000070
27,S7_D7 hbo,Condition[T.Speech],-2.079853,0.755616,-2.752528,0.005914
29,S7_D8 hbo,Condition[T.Speech],-3.930784,1.306914,-3.007683,0.002632
31,S8_D7 hbo,Condition[T.Speech],-2.652826,1.064348,-2.492441,0.012687


# LME MODEL

In [25]:
grp_results = df_roi.query("Condition in ['Control','Noise', 'Speech']")

for roi in ["Left", "Right", "Back", "Front", "Noise", "Speech", "Common", "OnlyNoise", "OnlySpeech"]:
    print(f"Running mixed model for {roi}...")
    subset = grp_results[(grp_results["Chroma"] == "hbo") & (grp_results["ROI"] == roi)].copy()

    # Ensure categorical variables
    subset["Condition"] = subset["Condition"].astype("category")
    subset["session"] = subset["session"].astype("category")
    
    subset["Condition"] = subset["Condition"].cat.reorder_categories(["Control", "Noise", "Speech"], ordered=True)


    # Fit mixed model: Condition + Session as fixed, ID as random intercept
    model = smf.mixedlm("theta ~ Condition + session + Condition*session", subset, groups=subset["Subject"])
    result = model.fit( method="powell")
    print(result.summary())


Running mixed model for Left...
                   Mixed Linear Model Regression Results
Model:                    MixedLM       Dependent Variable:       theta    
No. Observations:         96            Method:                   REML     
No. Groups:               16            Scale:                    4.1440   
Min. group size:          6             Log-Likelihood:           -206.3042
Max. group size:          6             Converged:                Yes      
Mean group size:          6.0                                              
---------------------------------------------------------------------------
                                 Coef.  Std.Err.   z    P>|z| [0.025 0.975]
---------------------------------------------------------------------------
Intercept                        -0.278    0.562 -0.494 0.621 -1.379  0.824
Condition[T.Noise]               -0.716    0.720 -0.994 0.320 -2.126  0.695
Condition[T.Speech]               0.396    0.720  0.550 0.583 -1.015  1.806

c:\Users\sarab\anaconda3\envs\mne\Lib\site-packages\statsmodels\regression\mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


                   Mixed Linear Model Regression Results
Model:                    MixedLM       Dependent Variable:       theta    
No. Observations:         96            Method:                   REML     
No. Groups:               16            Scale:                    2.6677   
Min. group size:          6             Log-Likelihood:           -189.2215
Max. group size:          6             Converged:                Yes      
Mean group size:          6.0                                              
---------------------------------------------------------------------------
                                 Coef.  Std.Err.   z    P>|z| [0.025 0.975]
---------------------------------------------------------------------------
Intercept                        -0.182    0.481 -0.378 0.705 -1.126  0.762
Condition[T.Noise]                0.217    0.577  0.376 0.707 -0.915  1.349
Condition[T.Speech]              -0.562    0.577 -0.974 0.330 -1.694  0.570
session[T.2]                   

# Left Hemisphere t-test

In [9]:
df_left= df_roi[df_roi["ROI"] == "Left"]

# control
df_left_control = df_left[df_left["Condition"] == "Control"]
df_left_control_hbo= df_left_control[df_left_control["Chroma"] == "hbo"]


# noise
df_left_noise = df_left[df_left["Condition"] == "Noise"]
df_left_noise_hbo= df_left_noise[df_left_noise["Chroma"] == "hbo"]


# speech
df_left_speech = df_left[df_left["Condition"] == "Speech"]
df_left_speech_hbo= df_left_speech[df_left_speech["Chroma"] == "hbo"]

# Now lets do a paired t-test with the theta values
from scipy.stats import ttest_rel
import pandas as pd

# Merge on Subject and session to ensure pairing
control_speech = pd.merge(
    df_left_control_hbo,
    df_left_speech_hbo,
    on=["Subject", "session"],
    suffixes=("_control","_speech")
)

# Run the paired t-test on the theta values
t_stat, p_value = ttest_rel(control_speech["theta_control"], control_speech["theta_speech"])

print("Paired t-test results (Control vs Speech) - Left Hemisphere:")
print(f"t-statistic = {t_stat:.4f}")
print(f"p-value = {p_value:.4f}")

# now for control and noise
control_noise = pd.merge(
    df_left_control_hbo,
    df_left_noise_hbo,
    on=["Subject", "session"],
    suffixes=("_control","_noise")
)
# Run the paired t-test on the theta values
t_stat, p_value = ttest_rel(control_noise["theta_control"], control_noise["theta_noise"])
print("Paired t-test results  (Control vs Noise) - Left Hemisphere:")
print(f"t-statistic = {t_stat:.4f}")
print(f"p-value = {p_value:.4f}")




Paired t-test results (Control vs Speech) - Left Hemisphere:
t-statistic = -0.0043
p-value = 0.9966
Paired t-test results  (Control vs Noise) - Left Hemisphere:
t-statistic = 1.0241
p-value = 0.3137


# Right Hemisphere t-test

In [219]:
# now for the right hemisphere
df_right= df_roi[df_roi["ROI"] == "Right"]
# control
df_right_control = df_right[df_right["Condition"] == "Control"]
df_right_control_hbo= df_right_control[df_right_control["Chroma"] == "hbo"]
# noise
df_right_noise = df_right[df_right["Condition"] == "Noise"]
df_right_noise_hbo= df_right_noise[df_right_noise["Chroma"] == "hbo"]
# speech
df_right_speech = df_right[df_right["Condition"] == "Speech"]
df_right_speech_hbo= df_right_speech[df_right_speech["Chroma"] == "hbo"]
# Merge on Subject and session to ensure pairing
control_speech = pd.merge(
    df_right_control_hbo,
    df_right_speech_hbo,
    on=["Subject", "session"],
    suffixes=("_control","_speech")
)
# Run the paired t-test on the theta values
t_stat, p_value = ttest_rel(control_speech["theta_control"], control_speech["theta_speech"])
print("Paired t-test results (Control vs Speech) - Right Hemisphere:")
print(f"t-statistic = {t_stat:.4f}")
print(f"p-value = {p_value:.4f}")
# now for control and noise
control_noise = pd.merge(
    df_right_control_hbo,
    df_right_noise_hbo,
    on=["Subject", "session"],
    suffixes=("_control","_noise")
)
# Run the paired t-test on the theta values
t_stat, p_value = ttest_rel(control_noise["theta_control"], control_noise["theta_noise"])
print("Paired t-test results  (Control vs Noise) - Right Hemisphere:")
print(f"t-statistic = {t_stat:.4f}")
print(f"p-value = {p_value:.4f}")


Paired t-test results (Control vs Speech) - Right Hemisphere:
t-statistic = 1.8730
p-value = 0.3122
Paired t-test results  (Control vs Noise) - Right Hemisphere:
t-statistic = 2.9330
p-value = 0.2092


# Back t-test

In [220]:
# now for the back of the head
df_back= df_roi[df_roi["ROI"] == "Back"]
# control
df_back_control = df_back[df_back["Condition"] == "Control"]
df_back_control_hbo= df_back_control[df_back_control["Chroma"] == "hbo"]
# noise
df_back_noise = df_back[df_back["Condition"] == "Noise"]
df_back_noise_hbo= df_back_noise[df_back_noise["Chroma"] == "hbo"]
# speech
df_back_speech = df_back[df_back["Condition"] == "Speech"]
df_back_speech_hbo= df_back_speech[df_back_speech["Chroma"] == "hbo"]
# Merge on Subject and session to ensure pairing
control_speech = pd.merge(
    df_back_control_hbo,
    df_back_speech_hbo,
    on=["Subject", "session"],
    suffixes=("_control","_speech")
)
# Run the paired t-test on the theta values
t_stat, p_value = ttest_rel(control_speech["theta_control"], control_speech["theta_speech"])
print("Paired t-test results (Control vs Speech) - Back Hemisphere:")
print(f"t-statistic = {t_stat:.4f}")
print(f"p-value = {p_value:.4f}")
# now for control and noise
control_noise = pd.merge(
    df_back_control_hbo,
    df_back_noise_hbo,
    on=["Subject", "session"],
    suffixes=("_control","_noise")
)
# Run the paired t-test on the theta values
t_stat, p_value = ttest_rel(control_noise["theta_control"], control_noise["theta_noise"])
print("Paired t-test results  (Control vs Noise) - Back Hemisphere:")
print(f"t-statistic = {t_stat:.4f}")
print(f"p-value = {p_value:.4f}")


Paired t-test results (Control vs Speech) - Back Hemisphere:
t-statistic = 2.1853
p-value = 0.2732
Paired t-test results  (Control vs Noise) - Back Hemisphere:
t-statistic = 11.1131
p-value = 0.0571


# Front t-test

In [221]:
# now for the front of the head
df_front= df_roi[df_roi["ROI"] == "Front"]
# control
df_front_control = df_front[df_front["Condition"] == "Control"]
df_front_control_hbo= df_front_control[df_front_control["Chroma"] == "hbo"]
# noise
df_front_noise = df_front[df_front["Condition"] == "Noise"]
df_front_noise_hbo= df_front_noise[df_front_noise["Chroma"] == "hbo"]
# speech
df_front_speech = df_front[df_front["Condition"] == "Speech"]
df_front_speech_hbo= df_front_speech[df_front_speech["Chroma"] == "hbo"]
# Merge on Subject and session to ensure pairing
control_speech = pd.merge(
    df_front_control_hbo,
    df_front_speech_hbo,
    on=["Subject", "session"],
    suffixes=("_control","_speech")
)
# Run the paired t-test on the theta values
t_stat, p_value = ttest_rel(control_speech["theta_control"], control_speech["theta_speech"])
print("Paired t-test results (Control vs Speech) - Front Hemisphere:")
print(f"t-statistic = {t_stat:.4f}")
print(f"p-value = {p_value:.4f}")
# now for control and noise
control_noise = pd.merge(
    df_front_control_hbo,
    df_front_noise_hbo,
    on=["Subject", "session"],
    suffixes=("_control","_noise")
)
# Run the paired t-test on the theta values
t_stat, p_value = ttest_rel(control_noise["theta_control"], control_noise["theta_noise"])
print("Paired t-test results  (Control vs Noise) - Front Hemisphere:")
print(f"t-statistic = {t_stat:.4f}")
print(f"p-value = {p_value:.4f}")


Paired t-test results (Control vs Speech) - Front Hemisphere:
t-statistic = 2.5492
p-value = 0.2380
Paired t-test results  (Control vs Noise) - Front Hemisphere:
t-statistic = 2.8886
p-value = 0.2122


# All optodes t-test

In [222]:
# now I want to consider all the channels as one ROI
df_all_Control= df_cha[df_cha["Condition"] == "Control"]
df_all_Control_hbo= df_all_Control[df_all_Control["Chroma"] == "hbo"]
# noise
df_all_noise = df_cha[df_cha["Condition"] == "Noise"]
df_all_noise_hbo= df_all_noise[df_all_noise["Chroma"] == "hbo"]
# speech
df_all_speech = df_cha[df_cha["Condition"] == "Speech"]
df_all_speech_hbo= df_all_speech[df_all_speech["Chroma"] == "hbo"]
# Merge on Subject and session to ensure pairing
control_speech = pd.merge(
    df_all_Control_hbo,
    df_all_speech_hbo,
    on=["Subject", "session"],
    suffixes=("_control","_speech")
)
# Run the paired t-test on the theta values
t_stat, p_value = ttest_rel(control_speech["theta_control"], control_speech["theta_speech"])
print("Paired t-test results (Control vs Speech) - All Channels:")
print(f"t-statistic = {t_stat:.4f}")
print(f"p-value = {p_value}")
# now for control and noise
control_noise = pd.merge(
    df_all_Control_hbo,
    df_all_noise_hbo,
    on=["Subject", "session"],
    suffixes=("_control","_noise")
)
# Run the paired t-test on the theta values
t_stat, p_value = ttest_rel(control_noise["theta_control"], control_noise["theta_noise"])
print("Paired t-test results  (Control vs Noise) - All Channels:")
print(f"t-statistic = {t_stat:.4f}")
print(f"p-value = {p_value}")

# now for noise vs speech
noise_speech = pd.merge(
    df_all_noise_hbo,
    df_all_speech_hbo,
    on=["Subject", "session"],
    suffixes=("_noise","_speech")
)
# Run the paired t-test on the theta values
t_stat, p_value = ttest_rel(noise_speech["theta_noise"], noise_speech["theta_speech"])
print("Paired t-test results (Noise vs Speech) - All Channels:")
print(f"t-statistic = {t_stat:.4f}")
print(f"p-value = {p_value}")




Paired t-test results (Control vs Speech) - All Channels:
t-statistic = 29.9237
p-value = 1.8933537288353043e-141
Paired t-test results  (Control vs Noise) - All Channels:
t-statistic = 29.0904
p-value = 1.0564730214348262e-135
Paired t-test results (Noise vs Speech) - All Channels:
t-statistic = 8.0601
p-value = 2.1371514858235485e-15


# Channel Specific T-tests

In [223]:
df_cha_channels= df_cha.copy()
df_cha_channels["ch_name_ind"] = df_cha["ch_name"].str.extract(r"^(\S+)")

df_cha_Control= df_cha_channels[df_cha_channels["Condition"] == "Control"]
df_cha_Control_hbo= df_cha_Control[df_cha_Control["Chroma"] == "hbo"]
# noise
df_cha_noise = df_cha_channels[df_cha_channels["Condition"] == "Noise"]
df_cha_noise_hbo= df_cha_noise[df_cha_noise["Chroma"] == "hbo"]

# speech
df_cha_speech = df_cha_channels[df_cha_channels["Condition"] == "Speech"]
df_cha_speech_hbo= df_cha_speech[df_cha_speech["Chroma"] == "hbo"]


In [224]:
optodes = ["S4_D2", "S6_D6", "S4_D3", "S5_D2", "S5_D3", "S5_D4", "S5_D5", 
           "S10_D9", "S10_D10", "S10_D11", "S10_D12", "S11_D11", "S11_D12", 
           "S6_D8", "S7_D6", "S7_D7", "S7_D8", "S8_D7", "S8_D8", "S9_D8", 
           "S1_D1", "S2_D1", "S3_D1", "S3_D2", "S12_D1"]

# Store results
results = []
# Perform paired t-tests per optode
for opt in optodes:
    df_c = df_cha_Control_hbo[df_cha_Control_hbo["ch_name_ind"] == opt]
    #print(df_c)
    df_n = df_cha_noise_hbo[df_cha_noise_hbo["ch_name_ind"] == opt]
    df_s = df_cha_speech_hbo[df_cha_speech_hbo["ch_name_ind"] == opt]

    for cond1, cond2, name1, name2 in [(df_c, df_n, "Control", "Noise"), 
                                       (df_c, df_s, "Control", "Speech"), 
                                       (df_n, df_s, "Noise", "Speech")]:
        merged = pd.merge(cond1[["ID", "theta"]], cond2[["ID", "theta"]], on="ID", suffixes=("_1", "_2"))
        if len(merged) > 1:
            t_stat, p_val = ttest_rel(merged["theta_1"], merged["theta_2"])
            results.append({
                "Optode": opt,
                "Comparison": f"{name1} vs {name2}",
                "n": len(merged),
                "t_statistic": t_stat,
                "p_value": p_val
            })

# Convert to DataFrame
results_df = pd.DataFrame(results)
# Add a column to indicate significance
results_df["Significant"] = results_df["p_value"] < 0.05

# Display the updated results
print(results_df)


    Optode         Comparison  n  t_statistic   p_value  Significant
0    S4_D2   Control vs Noise  2    -0.385428  0.765799        False
1    S4_D2  Control vs Speech  2     0.745967  0.591981        False
2    S4_D2    Noise vs Speech  2     5.926924  0.106409        False
3    S6_D6   Control vs Noise  2     5.101371  0.123231        False
4    S6_D6  Control vs Speech  2     1.669155  0.343623        False
..     ...                ... ..          ...       ...          ...
58   S3_D2  Control vs Speech  2     2.072557  0.286190        False
59   S3_D2    Noise vs Speech  2     4.029342  0.154867        False
60  S12_D1   Control vs Noise  2     1.610333  0.353776        False
61  S12_D1  Control vs Speech  2     1.374085  0.400506        False
62  S12_D1    Noise vs Speech  2     0.835560  0.556880        False

[63 rows x 6 columns]


In [225]:
# do a new dataframe with the results that are significant
significant_results = results_df[results_df["Significant"]]
significant_results['Optode'].unique()

array(['S5_D2', 'S5_D5', 'S10_D12', 'S8_D8', 'S9_D8', 'S1_D1', 'S3_D1'],
      dtype=object)

In [226]:
significant_results_control_noise = significant_results[significant_results["Comparison"] == "Control vs Noise"]
significant_results_control_speech = significant_results[significant_results["Comparison"] == "Control vs Speech"]
significant_results_noise_speech = significant_results[significant_results["Comparison"] == "Noise vs Speech"]

In [227]:
significant_results_control_noise['Optode'].unique()

array(['S5_D2', 'S5_D5', 'S9_D8', 'S1_D1'], dtype=object)

In [228]:
significant_results_control_speech['Optode'].unique()

array(['S10_D12', 'S8_D8', 'S3_D1'], dtype=object)

In [229]:
significant_results_noise_speech['Optode'].unique()

array([], dtype=object)